Analyze perturbation results
----------

This notebook looks at the result of a gene perturbation. 

In [1]:
%run task2.ipynb

INFO:datasets:PyTorch version 2.6.0 available.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)

  getattr(self, attr).index = value

  warnings.warn("Transforming to str index.", ImplicitModificationWarning)

  warnings.warn("Transforming to str index.", ImplicitModificationWarning)

INFO:helical.models.geneformer.model:Model finished initializing.
INFO:helical.models.geneformer.model:'gf-12L-95M-i4096-CLcancer' model is in 'eval' mode, on device 'cpu' with embedding mode 'cell'.
INFO:helical.models.geneformer.model:Processing data for Geneformer.
INFO:pyensembl.sequence_data:Loaded sequence dictionary from C:\Users\andre\AppData\Local\pyensembl\GRCh38\ensembl110\pyensembl\GRCh38\ensembl110\Cache\Homo_sapiens.GRCh38.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from C:\Users\andre\AppData\Local\pyensembl\GRCh38\ensembl110\pyensembl\GRCh38\ensembl110\Cache\Homo_sapiens.GRCh38.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loade

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

INFO:helical.models.geneformer.model:Successfully processed the data for Geneformer.
INFO:helical.models.geneformer.model:Started getting embeddings:


  0%|          | 0/20 [00:00<?, ?it/s]

INFO:helical.models.geneformer.model:Finished getting embeddings.
INFO:helical.models.geneformer.model:Processing data for Geneformer.
INFO:pyensembl.sequence_data:Loaded sequence dictionary from C:\Users\andre\AppData\Local\pyensembl\GRCh38\ensembl110\pyensembl\GRCh38\ensembl110\Cache\Homo_sapiens.GRCh38.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from C:\Users\andre\AppData\Local\pyensembl\GRCh38\ensembl110\pyensembl\GRCh38\ensembl110\Cache\Homo_sapiens.GRCh38.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from C:\Users\andre\AppData\Local\pyensembl\GRCh38\ensembl110\pyensembl\GRCh38\ensembl110\Cache\Homo_sapiens.GRCh38.pep.all.fa.gz.pickle
  adata.var["ensembl_id"] = pd.Series([None] * len(adata.var), index=adata.var.index)

  utils.warn_names_duplicates("var")

  utils.warn_names_duplicates("var")

INFO:helical.utils.mapping:Mapped 773 genes to Ensembl IDs from a total of 998 genes.
  data.var["gene_ids_collapsed"] = gen

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

INFO:helical.models.geneformer.model:Successfully processed the data for Geneformer.
INFO:helical.models.geneformer.model:Started getting embeddings:


  0%|          | 0/20 [00:00<?, ?it/s]

INFO:helical.models.geneformer.model:Finished getting embeddings.


In [2]:
# Calculate distances
dist = calculateDistance(pd.DataFrame(umapOriginal.embedding_, columns=['px','py']), pd.DataFrame(umapPerturbed.embedding_, columns=['px','py']))
# Perform knn analysis; as label use tum_norm to see if perturbed cells switch to the other cell type cluster
knnLabels, knnConfusion = knnClustering(gfEmbeddingOriginal, gfEmbeddingPerturbed, adataSubset.obs["tum_norm"])
# Get the scikit classification report
classReport = classification_report(adataSubset.obs["tum_norm"], knnLabels)

### Changes of UMAP embeddings

If a gene is a good target, it would be expected that the embedding of tumor cells changes more than the embedding of normal cells. This has two reasons:  
 1. If embeddings of tumor cells don't change, the perturbation did not affect them and no treatment affect would be expected.
 2. If embeddings of normal cells changes too much, treatemnt would be expected to cause side effects.

The following matrix shows the median distance in embedding for tumor and normal cells. Both values are very similar indicating that overexpression of ENSG00000188761 does not have a specific effect on tumor cells.

In [3]:
adataSubset.obs["dist"] = dist
adataSubset.obs[["tum_norm", "dist"]].groupby("tum_norm").agg("median")

,dist
tum_norm,
Normal,3.457562
Tumor,2.623294


### Difference in cell labels

Each cell was labeled as tumor or normal cell base on a knn clustering looking at the labels of the 5 closest cells. The confusion matrix shows the difference between the original label and the label predicted after perturbation. Good targets would be expected to have no strong effect on labelling of normal cells but do change predicted labels of tumors cells. 

The perturbation matrix shows that a similar number of tumor and normal cells are predicted to switch to the other class. This indicates that overexpression of ENSG00000188761 is not a viable treatment strategy.

In [4]:
print(knnConfusion)

[[85 15]
 [13 87]]


### Clustering metrics

The clustering report summarizes several metrics showing how good the agreement between original and predicted labels is. For good targets, 
it would be expected that the metrics show a high precision and recall for normal cells, indicating that most normal labels are stable. Additionally, 
it would be expected that the metrics show a high precision but lower recall for tumor cells, indicating that tumor cells switch label. But those cell 
identified as tumor cells are actual tumor cells.

The report below shows that both cell types are impacted in a highly similar manner by the perturbation, showing again that upregulation of ENSG00000188761
is not a viable strategy

In [5]:
print(classReport)

              precision    recall  f1-score   support

      Normal       0.87      0.85      0.86       100
       Tumor       0.85      0.87      0.86       100

    accuracy                           0.86       200
   macro avg       0.86      0.86      0.86       200
weighted avg       0.86      0.86      0.86       200



Summary
-----------

All results indicate that upregulation of ENSG00000188761 is not a viable treatment strategy for squamous cell carcinoma.